## Resultat för kurser (YH-utbildningar upp till 99 poäng)

### 2024

In [2]:
from pathlib import Path

import pandas as pd

DATA_PATH = "data"

current_dir = globals()["_dh"][0]  # CWD for jupyter
path_data = Path(current_dir) / DATA_PATH

In [3]:
# EXTRACT

df_2024 = pd.read_excel(path_data / "resultat-2024-for-kurser-inom-yh.xlsx", sheet_name=0, skiprows=0)
df_2024["Ansökningsomgång"] = 2024


# TRANSFORM

# Standardization of 'Beslut' and convert to bool
df_2024["Beslut"] = df_2024["Beslut"].map({"Beviljad": True, "Ej beviljad": False, "Avslag": False})
if df_2024["Beslut"].isna().sum() > 0:
    raise ValueError(f"Found {df_2024['Beslut'].isna().sum()} NaN values in 'Beslut'")


# Create bool column "Flera kommuner" from column "Kommun"
df_2024["Flera kommuner"] = False
df_2024.loc[df_2024["Kommun"] == 'Se "Lista flera kommuner"', "Flera kommuner"] = True


# Replace "Se \"Lista flera kommuner\"" with <NA>
df_2024["Kommun"] = df_2024["Kommun"].replace('Se "Lista flera kommuner"', pd.NA)
df_2024["Län"] = df_2024["Län"].replace('Se "Lista flera kommuner"', pd.NA)


# Sum and remove column "Antal beviljade platser start och slut 2024"
df_2024["Antal beviljade platser start 2024"] = (
    df_2024["Antal beviljade platser start 2024"] + df_2024["Antal beviljade platser start och slut 2024"]
)
df_2024 = df_2024.drop(columns=["Antal beviljade platser start och slut 2024"])


# Expand approved columns with year in column "Start"
years = [2024, 2025]
new_rows = []
for _, row in df_2024.iterrows():
    rows_added = False

    for year in years:
        column_name = f"Antal beviljade platser start {year}"
        if row[column_name] > 0:
            new_row = row.copy()
            new_row["Beviljade platser"] = row[column_name]
            new_row["Start"] = year
            new_rows.append(new_row)
            rows_added = True

    if not rows_added:
        new_row = row.copy()
        new_row["Beviljade platser"] = 0
        new_row["Start"] = years[0]
        new_rows.append(new_row)

df_2024 = pd.DataFrame(new_rows)


# Drop, rename, reorder
df_2024 = df_2024.drop(
    columns=[
        "Antal beviljade platser start 2024",
        "Antal beviljade platser start 2025",
        "FA-region",
        "Antal län",
        "Antal FA-regioner",
    ]
)

df_2024 = df_2024.rename(
    columns={
        "Anordnare namn": "Utbildningsanordnare",
        "Totalt antal beviljade platser": "Beviljade platser totalt",
    }
)

cols_order = [
    "Diarienummer",
    "Ansökningsomgång",
    "Beslut",
    "Utbildningsanordnare",
    "Utbildningsnamn",
    "Utbildningsområde",
    "YH-poäng",
    "Beviljade platser",
    "Beviljade platser totalt",
    "Start",
    "Län",
    "Kommun",
    "Antal kommuner",
    "Flera kommuner",
]
df_2024 = df_2024[cols_order]


# LOAD

df_2024.to_csv(path_data / "resultat-for-kurser-inom-yh-2024-beslut.csv", index=False)


# RESULT

df_2024.sample(10)

,Diarienummer,Ansökningsomgång,Beslut,Utbildningsanordnare,Utbildningsnamn,Utbildningsområde,YH-poäng,Beviljade platser,Beviljade platser totalt,Start,Län,Kommun,Antal kommuner,Flera kommuner
398,MYH 2024/1428,2024,True,YH Akademin AB,Hot och våld på arbetsplatsen,Hälso- och sjukvård samt socialt arbete,20,35,35,2024,Uppsala,Uppsala,1,False
45,MYH 2024/1054,2024,False,Trelleborgs kommun,Chefsstöd inom vård och omsorg,"Ekonomi, administration och försäljning",50,0,0,2024,Skåne,Trelleborg,1,False
161,MYH 2024/1177,2024,True,"Mariestads kommun, Dacapo Mariestad",Att tillvarata trädens nytta i staden- från pl...,"Lantbruk, djurvård, trädgård, skog och fiske",30,25,50,2024,Västra Götaland,Mariestad,1,False
815,MYH 2024/925,2024,False,"Kunskapsförbundet Väst, Vuxenutbildningen",Psykisk hälsa hos äldre,Hälso- och sjukvård samt socialt arbete,50,0,0,2024,Västra Götaland,Vänersborg,1,False
767,MYH 2024/863,2024,False,"Kunskapsförbundet Väst, Vuxenutbildningen",Socialadministration,"Ekonomi, administration och försäljning",99,0,0,2024,Västra Götaland,Vänersborg,1,False
431,MYH 2024/1462,2024,True,Montico Kompetensutveckling AB,Pensioner och försäkringar för lönekonsulter,"Ekonomi, administration och försäljning",15,30,60,2024,Kronoberg,Växjö,1,False
433,MYH 2024/1464,2024,False,Montico Kompetensutveckling AB,Produktionsledning inom tillverkande industri,Teknik och tillverkning,95,0,0,2024,Kronoberg,Växjö,1,False
597,MYH 2024/1628,2024,False,Technigo AB,UX Research,Data/IT,30,0,0,2024,<NA>,<NA>,3,True
696,MYH 2024/762,2024,False,Grit Academy,Avancerad JavaScript,Data/IT,60,0,0,2024,Skåne,Malmö,1,False
724,MYH 2024/803,2024,True,MedLearn AB,Ortodonti och bettfysiologi,Hälso- och sjukvård samt socialt arbete,30,35,35,2024,Stockholm,Stockholm,1,False


In [4]:
# EXTRACT

df_2024a = pd.read_excel(
    path_data / "resultat-2024-for-kurser-inom-yh.xlsx",
    sheet_name=0,
    skiprows=0,
    usecols=["Diarienummer", "Kommun", "Län", "Beslut"],
)

df_2024b = pd.read_excel(
    path_data / "resultat-2024-for-kurser-inom-yh.xlsx",
    sheet_name=2,
    skiprows=0,
    usecols=["Diarienummer", "Kommun", "Län"],
)


# TRANSFORM

# Map "Beslut" column to "Flera kommuner" rows
beslut_mapping = df_2024a.set_index("Diarienummer")["Beslut"].to_dict()
df_2024b["Beslut"] = df_2024b["Diarienummer"].map(beslut_mapping)


# Drop reffering rows
df_2024a = df_2024a[df_2024a["Kommun"] != 'Se "Lista flera kommuner"']


df_diarie_kommun = pd.concat([df_2024a, df_2024b], ignore_index=True)
df_diarie_kommun = df_diarie_kommun.drop_duplicates().sort_values(by="Diarienummer").reset_index(drop=True)


# Standardization of 'Beslut' and convert to bool
df_diarie_kommun["Beslut"] = df_diarie_kommun["Beslut"].map({"Beviljad": True, "Ej beviljad": False, "Avslag": False})
if df_diarie_kommun["Beslut"].isna().sum() > 0:
    raise ValueError(f"Found {df_diarie_kommun['Beslut'].isna().sum()} NaN values in 'Beslut'")


# LOAD

df_diarie_kommun.to_csv(path_data / "resultat-for-kurser-inom-yh-2024-diarie_kommun.csv", index=False)


# RESULT

df_diarie_kommun


,Diarienummer,Beslut,Kommun,Län
0,MYH 2024/1000,False,Stockholm,Stockholm
1,MYH 2024/1000,False,Boden,Norrbotten
2,MYH 2024/1000,False,Malmö,Skåne
3,MYH 2024/1001,False,Malmö,Skåne
4,MYH 2024/1001,False,Boden,Norrbotten
...,...,...,...,...
1197,MYH 2024/997,False,Nyköping,Södermanland
1198,MYH 2024/998,True,Nyköping,Södermanland
1199,MYH 2024/999,False,Malmö,Skåne
1200,MYH 2024/999,False,Boden,Norrbotten


In [5]:
df_antal = pd.read_excel(
    path_data / "resultat-2024-for-kurser-inom-yh.xlsx",
    sheet_name=0,
    skiprows=0,
    usecols=["Antal kommuner"],
)

sum_antal_kommuner = df_antal["Antal kommuner"].sum()

if sum_antal_kommuner != len(df_diarie_kommun):
    raise AssertionError(
        f"Sum of 'Antal kommuner' ({sum_antal_kommuner}) does not match count of rows in df_diarie_kommun ({len(df_diarie_kommun)})"
    )